<a href="https://colab.research.google.com/github/Alafiade/LeNet5-implementation/blob/main/Lenet_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries



In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn

Data Pipeline

In [2]:
Training_data = datasets.MNIST('/data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

Testing_data = datasets.MNIST('/data', train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),

                              ]))

100%|██████████| 9.91M/9.91M [00:00<00:00, 43.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.11MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.13MB/s]


In [3]:
len(Testing_data), len(Training_data)

(10000, 60000)

In [4]:


img_t,label = Training_data[99]
img_t.shape


torch.Size([1, 28, 28])

In [5]:
batch_size = 64
training_dataloader = DataLoader(Training_data, batch_size=batch_size, shuffle=True)
testing_dataloader = DataLoader(Testing_data, batch_size=batch_size, shuffle=False)

LeNet 5 architecture

In [25]:
class LeNet5(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2, stride=1),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=2,stride=2),

        nn.Conv2d(in_channels=6, out_channels=16,kernel_size=5, stride=1),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=2,stride=2),

        nn.Flatten(),
        nn.Linear(in_features=16*5*5, out_features=120),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=120, out_features=84),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=84, out_features=10),)




  def forward(self,x):
    logits = self.linear_relu_stack(x)
    return logits


model = LeNet5()













In [26]:

print(model)

LeNet5(
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.5, inplace=False)
    (10): Linear(in_features=120, out_features=84, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.5, inplace=False)
    (13): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [27]:
from torchsummary import summary
summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
              ReLU-9                  [-1, 120]               0
          Dropout-10                  [-1, 120]               0
           Linear-11                   [-1, 84]          10,164
             ReLU-12                   [-1, 84]               0
          Dropout-13                   [-1, 84]               0
           Linear-14                   

In [28]:
device = torch.device(device='cpu')

In [29]:
model = model.to(device)

Training the model

In [30]:
def train_loop(training_dataloader, model,loss_fn, optimizer,batch_size):
  size = len(training_dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(training_dataloader):
    pred = model(X)
    loss = loss_fn(pred,y)


    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    if batch % 100 == 0:
      loss, current = loss.item(), batch* batch_size + len(X)
      print(f'loss : {loss:>7f} [{current:>5d}/{size:>5d}]')






def test_loop(testing_dataloader, model, loss_fn):
    model.eval()
    size = len(testing_dataloader.dataset)
    num_batches = len(testing_dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in testing_dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_dataloader, model, loss_fn, optimizer, batch_size=64)
    test_loop(testing_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss : 2.290863 [   64/60000]
loss : 0.848607 [ 6464/60000]
loss : 0.344769 [12864/60000]
loss : 0.438794 [19264/60000]
loss : 0.445956 [25664/60000]
loss : 0.238465 [32064/60000]
loss : 0.266884 [38464/60000]
loss : 0.385404 [44864/60000]
loss : 0.251906 [51264/60000]
loss : 0.165425 [57664/60000]
Test Error: 
 Accuracy: 95.7%, Avg loss: 0.416613 

Epoch 2
-------------------------------
loss : 0.119476 [   64/60000]
loss : 0.236938 [ 6464/60000]
loss : 0.160512 [12864/60000]
loss : 0.311286 [19264/60000]
loss : 0.066442 [25664/60000]
loss : 0.100177 [32064/60000]
loss : 0.264825 [38464/60000]
loss : 0.158646 [44864/60000]
loss : 0.079174 [51264/60000]
loss : 0.319317 [57664/60000]
Test Error: 
 Accuracy: 94.9%, Avg loss: 0.406462 

Epoch 3
-------------------------------
loss : 0.078060 [   64/60000]
loss : 0.138054 [ 6464/60000]
loss : 0.102183 [12864/60000]
loss : 0.233705 [19264/60000]
loss : 0.026895 [25664/60000]
loss : 0.076844 [32064/600